### Import libs

In [ ]:
import matplotlib
import numpy as np
%matplotlib notebook
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import math
from IPython.display import HTML
import pandas as pd

plt.rcParams['animation.embed_limit'] = 2**128
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

LVC_path = '/misc/home/u1220/analysis_jupyter'
import sys
sys.path.remove('/home/u1220/lvd_aniso/jupiter_books') if '/home/u1220/lvd_aniso/jupiter_books' in sys.path else None
sys.path.append(LVC_path) if not LVC_path in sys.path else None

from LVC.LVCcase_v2 import LVCcase

plt.style.use('LVC')

### Load data

In [ ]:
%store -r proj_for_trade
proj_for_trade = r'/misc/home/u1220/lvd_aniso/mol_a_2nd/results_TP06/id_9/t1=60000pace_start=5000Ttarget=227.0Diffuz2=0.154'
print(proj_for_trade)
proj = LVCcase(project_folder=proj_for_trade, auto_load=True, plot=plt)

proj.read_input(verbose=False)

#### udat

In [ ]:
proj.load_udat(verbose=True)
proj.get_frames()
frames = proj.frames

### Tips

In [ ]:
tips_fig = figure(num=None, figsize=(14, 4), dpi=80, facecolor='w')

proj.draw_tipXY(show_stim=False, legend=False)
# plt.xlim(0, 1000)
plt.show()

### Calc period by tips

In [ ]:
times, periods = proj.calculate_periods_by_tips(start=20000, end=30000, u_level=47, coordinate='y', show_plot=True)

figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
plt.scatter(times, periods, s=7)
plt.grid()
plt.title("SW periods using tip coordinates")
plt.show()
print(np.average(periods))

### Draw frame

In [ ]:
frame_fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
time=44
proj.draw_frame(plot=plt, N=int(time/proj.dT), show_tips=True, set_title=True, vmax=25)

### Animation

In [ ]:
%matplotlib inline
fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
proj.setup_animation(K=2, first=25000, last=30000, show_tips=True, draw_trace=False, axis_auto_scale=False, vmax=25)

proj.generate_animation(fig)

# proj.animation.save("/misc/home1/u1220/lvc/analysis_jupyter/_trash/SW_formation.gif",  writer='imagemagick', fps=15)
# proj.save_animation(filename='sl14norm_T=232.mp4', overwrite=False, fps=10, folder='/misc/home1/u1220/lvc/analysis_jupyter/_trash/')
# proj.save_animation(generate_filename=True)

%matplotlib notebook
HTML(proj.animation.to_jshtml())

### Stimtable

In [ ]:
pd.DataFrame(proj.stimtable)

### Potential at point

In [ ]:
x, y = (50,50)

x_ind = int(x/(proj.mult_x*proj.drx))
y_ind = int(y/(proj.mult_y*proj.dry))

proj.load_udat()
time_axis = np.arange(0, proj.frames_number) * proj.dT
        
potential_fig = figure(num=None, figsize=(14, 4), dpi=80, facecolor='w')
plt.plot(list(time_axis), list(proj.get_point_in_time(x_ind,y_ind)), linewidth=1, color='red')
plt.title('Potential at point (%.2f, %.2f), indices (%d, %d)' % 
          (x_ind * proj.drx * proj.mult_x, y_ind * proj.dry * proj.mult_y, x_ind, y_ind))
plt.grid(True)
# plt.xlim(10000,15000)
# plt.ylim(-100,110)

plt.show()

### Periods by AP

In [ ]:
start_t, end_t, u_level = (500, 2900, 50)
start_t_ind, end_t_ind = int(start_t/proj.dT), int(end_t/proj.dT)

times, periods = proj.calculate_periods(x=x_ind, y=y_ind, start=start_t_ind, end=end_t_ind, u_level=u_level)

figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
plt.scatter(times, periods, s=7)
plt.grid()
plt.title("Periods at point (%.2f, %.2f) on interval [%.2f, %.2f]. Potential level = %.2f." % 
          (x_ind*proj.drx*proj.mult_x, y_ind*proj.dry*proj.mult_y, start_t_ind*proj.dT, end_t, u_level))
plt.show()
print(np.average(periods))

### APD

In [ ]:
# get APD_X
x, y, start_t, end_t = (10, 10, 500, 60000)

x_ind = int(x/(proj.mult_x*proj.drx))
y_ind = int(y/(proj.mult_y*proj.dry))
start_t_ind = int(start_t/proj.dT)
end_t_ind = int(end_t/proj.dT)

times, apds = proj.calculate_APD_by_percentage(x=x_ind, y=y_ind, start=start_t_ind, end=end_t_ind, APD_percent=.9)

figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
plt.scatter(times, apds, s=7)
plt.grid()
# plt.title("APD_%d at point (%.2f, %.2f) in interval [%.2f, %.2f]. Potential level = %.2f." % 
#           (APD_level, x_ind*proj.drx*proj.mult_x, y_ind*proj.dry*proj.mult_y, start_t_ind*proj.dT, proj.calc_time, APD_level))
# plt.xlim(0,8000)
# plt.ylim(200,230)
print("Avg APD90: %.2f, Median APD90: %.2f" % (np.average(apds), np.median(apds)))
plt.show()

### Trajectory

In [ ]:
traject_fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
proj.load_tips()
tips = proj.tips[proj.tips[:,0] < 1000]

ax = plt.gca()
ax.set_axisbelow(True)

plt.xlim(xmax = proj.nx * proj.drx, xmin = 0)
plt.ylim(ymax = proj.ny * proj.dry, ymin = 0)
plt.scatter(tips[:,2], tips[:,3], color='red', s=1)
plt.grid(True)

### Wavelength

In [ ]:
# get wavelength
y, rest_level, frame_number = (50, 7, 400)
y_ind = int(y/(proj.mult_y*proj.dry))

frame_fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
frame = proj.frames[frame_number]
plt.imshow(frame, origin='lower', interpolation='none', extent=[0, proj.nx * proj.drx, 0, proj.ny * proj.dry], vmin=np.amin(frames), vmax=25)
plt.plot([0, proj.nx*proj.drx], [(y_ind)*proj.dry*proj.mult_y]*2, color='black')
plt.title('Spiral wave motion at %.2f ms' % (frame_number*proj.dT))
plt.show()

xs, wls = proj.calculate_wavelength(y=y_ind, u_level=rest_level, frame_number=frame_number)
print(wls)

figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
plt.scatter(xs, wls, s=7)
plt.grid()
plt.show()

### 1D vs 2D comparison

In [ ]:
# for 1D vs 2D comparison

layer_num=0
layer_in_time = [np.array(frame[layer_num]) for frame in proj.frames]

np.savetxt('/misc/home1/u1220/lvc/analysis_jupyter/_trash/TP06sl14norm_strip_to_cabel.dat', layer_in_time, fmt="%d")